## An introduction to Turing 

[Turing](https://github.com/yebai/Turing.jl) is a general-purpose probabilistic programming language based on Julia.

Here we describe how to install Julia and Turing and run a very simple probabilistic program. 

### Prepration

Install Julia from http://julialang.org/downloads/ and update packages

Install the stable version of Turing 

In [ ]:
Pkg.add("Turing")

or alternatively you can install the development of Turing (not recommended)

In [ ]:
# Pkg.clone("https://github.com/yebai/Turing.jl")
# Pkg.checkout("Turing", "master")

#### Create your own model
A probabilistic program is Julia code wrapped in a @model macro. It can use arbitrary Julia code, but to ensure correctness of inference it should not have external effects or modify global state. 

You can create your own @model using any distribution within the Distributions package. The list of such distributions supported is huge: 
http://distributionsjl.readthedocs.io/en/latest/ 

In [2]:
using Turing, Distributions

@model gdemo(x) = begin
  s ~ InverseGamma(2,3)
  m ~ Normal(0,sqrt(s))
  x[1] ~ Normal(m, sqrt(s))
  x[2] ~ Normal(m, sqrt(s))
  return s, m
end


 Assume - `s` is a parameter
 Assume - `m` is a parameter
 Observe - `x` is an observation


gdemo (generic function with 4 methods)

## Inference

In [4]:
# Sample and print.
c = @sample(gdemo([1.5, 2]), PG(50,300))
s = MambaChains(c)
describe(s)

# Compute analytical solution. Requires `ConjugatePriors` package.
# exact = posterior(NormalInverseGamma(0,1,2,3), Normal, [1.5,2.0])
# println("Exact: m = $(mean(exact)[1]), s = $(mean(exact)[2])")

[PG]: Finshed within 1.9410021305084229 seconds
Iterations = 1:300
Thinning interval = 1
Chains = 1
Samples per chain = 300

Empirical Posterior Estimates:
     Mean       SD     Naive SE      MCSE    ESS
m 1.1608222 0.7858986 0.04537388 0.029122925 300
s 2.0584310 1.9115138 0.11036130 0.082673365 300

Quantiles:
      2.5%       25.0%     50.0%     75.0%     97.5%  
m -0.37338789 0.68540733 1.1395728 1.6602254 2.6820722
s  0.61467613 1.03650586 1.4995534 2.2656389 6.9556917



In [14]:
c2 =  @sample(gdemo([1.5, 2]), HMC(1000, 0.3, 10));
s2 = MambaChains(c2)
describe(s2)

[HMC]: Finshed with accept rate = 0.971 within 4.9229090213775635 seconds
Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Empirical Posterior Estimates:
     Mean       SD     Naive SE      MCSE     ESS
m 1.1622141 0.8644466 0.02733620 0.020580453 1000
s 2.0567298 1.9582019 0.06192378 0.053387299 1000

Quantiles:
     2.5%      25.0%     50.0%     75.0%     97.5%  
m -0.7271658 0.6804992 1.2129055 1.7036967 2.7719666
s  0.5410996 1.0441755 1.5057669 2.3300369 7.1319002

